## Analysis of Hyperparameters and Comparison between Q-Learning and SARSA Algorithm 

### Imports

In [1]:
import gymnasium as gym
import numpy as np
import time
import random
from IPython.display import clear_output
import matplotlib.pyplot as plt
import scipy.stats as st
import seaborn as sns
import pandas as pd
sns.set_style("darkgrid")
%matplotlib qt5

### Q-Learning Algorithm

In [2]:
def train_q_learn(env, learning_rate, discount_rate, num_episodes, max_steps_per_episode, exploration_rate, max_exploration_rate, min_exploration_rate, exploration_decay_rate):
    '''
    Training a q-table with the Q-Learning algorithm
    '''
    rewards_all_episodes = []
    steps_all_episodes = []
    # Initialize the q-table with zeros
    q_table = np.zeros([env.observation_space.n, env.action_space.n])

    # Random generator
    rng =np.random.default_rng()

    # Start training for num_episodes
    for episode in range(num_episodes):
        # Reset the environment and initialize counters
        state, info = env.reset()
        step = 0
        rewards_current_episode  = 0
        
        for step in range(max_steps_per_episode):
            # Choose an action in the current state
            action = eps_greedy(env, q_table, state, exploration_rate)

            # Take the action and get the new state and reward
            new_state, reward, done, truncated, info = env.step(action)

            # Update with Bellman Equation Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
            q_table[state, action] = q_table[state, action] +  learning_rate * (reward + discount_rate * np.max(q_table[new_state, :] - q_table[state, action]))
    
            # Count rewards and steps
            rewards_current_episode  += reward
            step += 1
            
            # Our new state is state
            state = new_state
            
            # If done: finish episode
            if done or truncated: 
                break
            
        # Reduce exploration
        exploration_rate = min_exploration_rate + (max_exploration_rate - min_exploration_rate)*np.exp(-exploration_decay_rate*episode) 
        rewards_all_episodes.append(rewards_current_episode)
        steps_all_episodes.append(step)
    return q_table, rewards_all_episodes, steps_all_episodes
    


### SARSA Algorithm

In [12]:
def train_sarsa(env, learning_rate, discount_rate, num_episodes, max_steps_per_episode, exploration_rate, max_exploration_rate, min_exploration_rate, exploration_decay_rate):
    '''
    Training a q-table with the SARSA algorithm
    '''
    rewards_all_episodes = []
    steps_all_episodes = []
    # Initialize the q-table with zeros
    q_table = np.zeros([env.observation_space.n, env.action_space.n])

    # Random generator
    rng =np.random.default_rng()

    # Start training for num_episodes
    for episode in range(num_episodes):
        # Reset the environment and initialize counters
        state, info = env.reset()
        step = 0
        rewards_current_episode  = 0
        # Choose an action in the current state
        action = eps_greedy(env, q_table, state, exploration_rate)

        for step in range(max_steps_per_episode):
            
            # Take the action and get the new state and reward
            new_state, reward, done, truncated, info = env.step(action)

            # Choose an action in for the new state
            new_action = eps_greedy(env, q_table, new_state, exploration_rate)

            # Update using Bellman Equation Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * Q(s',a') - Q(s,a)]
            q_table[state, action] = q_table[state, action]  +  learning_rate * (reward + discount_rate * q_table[new_state, new_action] - q_table[state, action])
    
            # Count rewards and steps
            rewards_current_episode  += reward
            step += 1
            
            # Our new state is state
            # Our new action is action
            state = new_state
            action = new_action
            
            # If done: finish episode
            if done or truncated: 
                break
            
        # Reduce exploration
        exploration_rate = min_exploration_rate + (max_exploration_rate - min_exploration_rate)*np.exp(-exploration_decay_rate*episode) 
        rewards_all_episodes.append(rewards_current_episode)
        steps_all_episodes.append(step)

        
    return q_table, rewards_all_episodes, steps_all_episodes
    


### Greedy and non Greedy Function

In [4]:
def greedy(q_table, state):
    '''
    Greedy Policy

    Returns the index of the highest action value for current state
    '''
    return np.argmax(q_table[state,:])

def eps_greedy(env, q_table, state, exploration_rate):
    '''
    Epsilon-Greedy Policy

    Generates a random number between 0 and 1 and checks if the current exploration rate (epsilon) is lower.
    In case it is lower it returns the index of the highest action value for current state.
    Otherwise it chooses exploring and takes a random action
    '''
    exploration_rate_threshold = random.uniform(0, 1)
    if exploration_rate_threshold > exploration_rate:
        return greedy(q_table, state)
    else:
        return env.action_space.sample()


# Evaluation

### Comparing Algorithms

#### Histograms for Rewards and Steps

In [5]:
def eval_training(rewards_all_episodes, steps_all_episodes, num_episodes):
    '''
    Evaluates the training by calculating the means and plotting histograms of Rewards and Steps needed during training for 10 different subsets
    '''

    # Splits both arrays into multiple subarrays
    rewards_per_x_episodes = np.split(np.array(rewards_all_episodes),num_episodes/(num_episodes/10))
    steps_per_x_episodes = np.split(np.array(steps_all_episodes),num_episodes/(num_episodes/10))
    count = num_episodes/10

    # Prints average rewards and steps for ten sub intervalls 
    print("********Average reward and steps per ",num_episodes/10 ," episodes********\n")
    for r, s in zip(rewards_per_x_episodes, steps_per_x_episodes):
        print(count, ": Reward = ", str(round(sum(r/(num_episodes/10)), 2)), ", Steps = ", str(round(sum(s/(num_episodes/10)), 2)))
        count += num_episodes/10

    # initialize Reward Histgram Plot
    fig, axes = plt.subplots(2, 5, figsize=(25, 10))
    fig.suptitle('Reward Histograms per ' + str(int(num_episodes/10)) + ' episodes during training')

    # Plot rewards
    for i in range(len(rewards_per_x_episodes)):
        count += num_episodes/10
        sns.histplot(ax=axes[int(5%(i+1)/5), i-int(5%(i+1)/5)*5], x = rewards_per_x_episodes[i])
        axes[int(5%(i+1)/5), i-int(5%(i+1)/5)*5].set_title( str(len(rewards_per_x_episodes[i])*(i+1)-int(num_episodes/10)) + " - " + str(len(rewards_per_x_episodes[i])*(i+1)))

    # initialize Step Histgram Plot
    fig, axes = plt.subplots(2, 5, figsize=(25, 10))
    fig.suptitle('Step Histograms per ' + str(int(num_episodes/10)) + ' episodes during training')

    # Plot Steps
    for i in range(len(steps_per_x_episodes)):
        count += num_episodes/10
        sns.histplot(ax=axes[int(5%(i+1)/5), i-int(5%(i+1)/5)*5], x = steps_per_x_episodes[i])
        axes[int(5%(i+1)/5), i-int(5%(i+1)/5)*5].set_title( str(len(rewards_per_x_episodes[i])*(i+1)-int(num_episodes/10)) + " - " + str(len(rewards_per_x_episodes[i])*(i+1)))
    

    


#### Lineplots for Rewards and Steps

In [6]:
def compare_algorithm(x,y,plt_type):
    '''
    Function which compares the different algorithms 
    '''
    x_series = pd.Series(x)
    x_series = x_series.rolling(15).mean()
    plt.plot(x_series)

    y_series = pd.Series(y)
    y_series = y_series.rolling(15).mean()
    plt.plot(y_series)

    plt.legend(["Q-Learning", "SARSA"])
    plt.xlabel("Episodes")
    plt.ylabel(plt_type)

    # plt.yscale("symlog")
    plt.show()

### Comparing Parameters

#### Lineplots for Rewards

In [19]:
def compare_params(x,y):
    '''
    Function which compares the same algorithm with different parameters
    '''
    step_series = pd.Series(x)
    step_series = step_series.rolling(15).mean()
    plt.plot(step_series)

    step_series = pd.Series(y)
    step_series = step_series.rolling(15).mean()
    plt.plot(step_series)

    #plt.title("Sports Watch Data")
    #plt.legend(["discount_rate = 0.99", "discount_rate = 0.2", "learning_rate  = 0.8"])
    plt.xlabel("Episodes")
    plt.ylabel("Reward")

    #plt.yscale("symlog")
    plt.show()

### Visualisation on what the agent does after training

In [8]:
def visualise(env, q_table, max_steps_per_episode, num_vis_episodes=3):
    '''
    Function which simulates and displays the game a certain amount of times.
    '''
    # Loop over episodes to visualise
    for episode in range(num_vis_episodes):
        state, _ = env.reset()
        print("********EPISODE ", episode+1, "********\n\n\n\n")
        time.sleep(1)
        
        # Render environment for each step and sleep 0.3 seconds
        for step in range(max_steps_per_episode):
            clear_output(wait=True)
            print(env.render())
            time.sleep(0.3)

            action = np.argmax(q_table[state,:])
            new_state, reward, done, truncated, _ = env.step(action)
            if done or truncated: 
                clear_output(wait=True)
                print(env.render())
                if reward > 0:
                    print("****You reached the goal!****")
                    time.sleep(3)
                else:
                    print("****Something bad happened!****")
                    time.sleep(3)
                    clear_output(wait=True)
                break
            state = new_state
        

### Calculate Metrics for evaluation after training

In [9]:
def eval_test(env, num_test_episodes, max_steps_per_episode, q_table):
    '''
    Function which tests the trained q-table without using exploration
    '''
    # initialize counters
    total_steps, total_penalties, total_reward, total_errors = 0, 0, 0, 0

    # Loop over episodes to analyze
    for episode in range(num_test_episodes):
            state, _ = env.reset()

            # Play the game until the agent dies or succeeds and calculate metrics
            for step in range(max_steps_per_episode):
                total_steps += 1
                action = np.argmax(q_table[state,:])
                new_state, reward, done, truncated, _ = env.step(action)
                total_reward += reward
                if reward == -10:
                    total_penalties += 1

                if done or truncated: 
                    if reward > 0:
                        break
                    else:
                        total_errors += 1
                        break
                
                if step == max_steps_per_episode  - 1:
                    total_errors += 1

                state = new_state
    
    # Print Metrics
    print("Avg Steps : ", str(round(total_steps/num_test_episodes, 2)))
    print("Total Penalties: ", str(int(total_penalties)))
    print("Avg Reward : ", str(round(total_reward/num_test_episodes, 2)))
    print("Total Errors: ", str(int(total_errors)))

### Select Environment and Setup Hyperparamters

In [32]:
# Environment Selection and Hyperparameters


env = gym.make("Taxi-v3", render_mode='ansi').env
# env = gym.make("CliffWalking-v0", render_mode="ansi")
# env = gym.make('FrozenLake-v1', desc=None, map_name="4x4", is_slippery=True, render_mode='ansi').env 

params1 = [
    0.1,    #learning rate
    0.99,   #discount rate
    3000,   #num of Episodes
    1000,   #max steps per Episode
    1,      #exploration rate
    1,      #max explaration rate
    0.05,   #min exploration rate
    0.005,  #exploration decay rate
]

params2 = [
    0.1,    #learning rate
    0.99,   #discount rate
    1300,   #num of Episodes
    1000,   #max steps per Episode
    1,      #exploration rate
    1,      #max explaration rate
    0.01,   #min exploration rate
    0.005,  #exploration decay rate
]

### Train different q-Tables with different Parameters and Algorithms

In [31]:
q_table1, reward_list1, step_list1 = train_q_learn(env, *params1)
s_q_table1, s_reward_list1, s_step_list1 = train_sarsa(env, *params1)
q_table2, reward_list2, step_list2 = train_q_learn(env, *params2)
s_q_table2, s_reward_list2, s_step_list2 = train_sarsa(env, *params2)

### Compare Algorithms

In [33]:
compare_algorithm(reward_list1, s_reward_list1,  "Rewards")

In [34]:
compare_algorithm(step_list1, s_step_list1,  "Steps")

### Compare Parameters

In [35]:
compare_params(reward_list1, reward_list2)

### Evaluate Training with metrics + histograms

In [36]:
eval_training(reward_list1, step_list1, params1[2])
eval_training(s_reward_list1, s_step_list1, params1[2])

********Average reward and steps per  300.0  episodes********

300.0 : Reward =  -859.69 , Steps =  292.95
600.0 : Reward =  -58.04 , Steps =  53.21
900.0 : Reward =  -10.13 , Steps =  25.58
1200.0 : Reward =  1.98 , Steps =  16.71
1500.0 : Reward =  3.85 , Steps =  15.23
1800.0 : Reward =  4.44 , Steps =  14.46
2100.0 : Reward =  4.73 , Steps =  14.23
2400.0 : Reward =  5.23 , Steps =  13.73
2700.0 : Reward =  5.69 , Steps =  13.9
3000.0 : Reward =  5.21 , Steps =  13.69
********Average reward and steps per  300.0  episodes********

300.0 : Reward =  -939.89 , Steps =  322.76
600.0 : Reward =  -66.74 , Steps =  58.58
900.0 : Reward =  -21.84 , Steps =  34.83
1200.0 : Reward =  -3.47 , Steps =  21.41
1500.0 : Reward =  2.73 , Steps =  16.08
1800.0 : Reward =  4.01 , Steps =  15.01
2100.0 : Reward =  4.31 , Steps =  14.68
2400.0 : Reward =  4.46 , Steps =  14.47
2700.0 : Reward =  5.44 , Steps =  14.0
3000.0 : Reward =  5.3 , Steps =  14.02


### Visualise environment and agent in action

In [37]:
visualise(env, s_q_table1, params1[3])

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

****You reached the goal!****


### Calculate Metrics for test scenario without exploration

In [39]:
eval_test(env, num_test_episodes=100, max_steps_per_episode=params1[3], q_table=q_table1)

Avg Steps :  13.45
Total Penalties:  0
Avg Reward :  7.55
Total Errors:  0
